<a href="https://colab.research.google.com/github/OdysseusPolymetis/colabs_for_nlp/blob/main/4_ner_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair
!pip install stanza

In [ ]:
import stanza
import numpy as np
from tqdm import tqdm
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load("UGARIT/flair_grc_bert_ner")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/464M [00:00<?, ?B/s]

2024-11-20 13:44:02,583 SequenceTagger predicts: Dictionary with 15 tags: O, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-LOC, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
sentence = Sentence('ταῦτα εἴπας ὁ Ἀλέξανδρος παρίζει Πέρσῃ ἀνδρὶ ἄνδρα Μακεδόνα ὡς γυναῖκα τῷ λόγῳ · οἳ δέ , ἐπείτε σφέων οἱ Πέρσαι ψαύειν ἐπειρῶντο , διεργάζοντο αὐτούς .')
tagger.predict(sentence)
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
with open('/content/odyssee_integrale.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
stanza.download('grc')
nlp = stanza.Pipeline(lang='grc', processors='tokenize,lemma')

In [ ]:
doc=nlp(text)

from collections import defaultdict
import numpy as np

# Initialisation du dictionnaire pour la matrice de cooccurrence
cooccurrence_dict = defaultdict(lambda: defaultdict(int))

for sentence in doc.sentences:
    sentence_text = sentence.text

    # Prédiction NER avec Flair
    ner_sentence = Sentence(sentence_text)
    tagger.predict(ner_sentence)

    # Extraction des entités NER de type PER
    ner_entities = [(entity.text, entity.start_position, entity.end_position) for entity in ner_sentence.get_spans('ner') if entity.get_label('ner').value == 'PER']

    # Pour chaque entité NER PER, trouver les cooccurrences dans la fenêtre de 20 mots
    for i, (entity_text_i, start_i, end_i) in enumerate(ner_entities):
        lemma_i = ' '.join([token.lemma for token in sentence.words if entity_text_i in token.text])

        for j, (entity_text_j, start_j, end_j) in enumerate(ner_entities):
            if i != j and abs(start_i - start_j) <= 20:
                lemma_j = ' '.join([token.lemma for token in sentence.words if entity_text_j in token.text])
                cooccurrence_dict[lemma_i][lemma_j] += 1

# Conversion du dictionnaire en matrice
entities = list(cooccurrence_dict.keys())
matrix_size = len(entities)
cooccurrence_matrix = np.zeros((matrix_size, matrix_size), dtype=int)

for i, entity_i in enumerate(entities):
    for j, entity_j in enumerate(entities):
        cooccurrence_matrix[i, j] = cooccurrence_dict[entity_i][entity_j]

# Affichage de la matrice
print(cooccurrence_matrix)

In [ ]:
import networkx as nx

G = nx.Graph()

# Ajout des nœuds
for i, entity in enumerate(entities):
    G.add_node(i, label=entity)

# Ajout des arêtes
for i, row in enumerate(cooccurrence_matrix):
    for j, weight in enumerate(row):
        if weight > 0 and i != j:
            G.add_edge(i, j, weight=weight)

# Exportation en GEXF
nx.write_gexf(G, "network.gexf")